# Additional End of week Exercise - week 2

Now use everything you've learned from Week 2 to build a full prototype for the technical question/answerer you built in Week 1 Exercise.

This should include a Gradio UI, streaming, use of the system prompt to add expertise, and the ability to switch between models. Bonus points if you can demonstrate use of a tool!

If you feel bold, see if you can add audio input so you can talk to it, and have it respond with audio. ChatGPT or Claude can help you, or email me if you have questions.

I will publish a full solution here soon - unless someone beats me to it...

There are so many commercial applications for this, from a language tutor, to a company onboarding solution, to a companion AI to a course (like this one!) I can't wait to see your results.

In [ ]:
import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from scraper import fetch_website_links, fetch_website_contents
from openai import OpenAI
import gradio as gr
import sqlite3
import base64
from io import BytesIO
from PIL import Image
import re

In [ ]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'
openai = OpenAI()

DB = "wiedza.db"

In [ ]:
# set up environment

OLLAMA_BASE_URL = "http://localhost:11434/v1"

ollama = OpenAI(base_url=OLLAMA_BASE_URL, api_key='ollama')

In [ ]:
system_message = """
You are a technical assistant for residential construction knowledge.
Answer questions using the provided database and tools whenever factual information is required.
Give short, clear, and professional answers (maximum 5 sentences).
Do not guess or invent information.
If the database does not contain the answer, say explicitly that the information is not available.
When numerical values or technical rules are mentioned, they must come from the database.
"""

In [ ]:
def get_information(query):
    print(f"DATABASE TOOL CALLED: Getting information for {query}", flush=True)

    q = (query or "").strip()
    if not q:
        return "No query provided."

    q_low = q.lower()

    synonyms = {
        "electrical box": "puszka",
        "electrical boxes": "puszka",
        "electrical": "elektryka",
        "spacing": "rozstaw",
        "transport anchors": "haki transportowe",
        "transport anchor": "hak transportowy",
        "transport hooks": "haki transportowe",
        "hooks": "haki",
        "requirements": "wymagania",
    }
    for k, v in synonyms.items():
        q_low = q_low.replace(k, v)

    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()

        # 1) Komponenty
        cursor.execute(
            """
            SELECT name, manufacturer, category, COALESCE(code,''), COALESCE(notes,'')
            FROM component
            WHERE lower(name) LIKE ? OR lower(COALESCE(code,'')) = ?
            LIMIT 1
            """,
            (f"%{q_low}%", q_low)
        )
        comp = cursor.fetchone()
        if comp:
            name, manufacturer, category, code, notes = comp
            code_part = f", code: {code}" if code else ""
            return f"{name} (category: {category}, manufacturer: {manufacturer}{code_part}). {notes}".strip()

        # 2) Chunks wiedzy
        cursor.execute(
            """
            SELECT title, content, page_from
            FROM knowledge_chunk
            WHERE lower(title) LIKE ?
               OR lower(content) LIKE ?
               OR lower(COALESCE(tags,'')) LIKE ?
               OR lower(COALESCE(site_scope,'')) LIKE ?
            ORDER BY page_from
            LIMIT 3
            """,
            (f"%{q_low}%", f"%{q_low}%", f"%{q_low}%", f"%{q_low}%")
        )
        chunks = cursor.fetchall()
        if chunks:
            return "\n\n".join([f"{t} (p.{p}): {c}" for (t, c, p) in chunks])

        # 3) Liczby / rozstawy / odstępy (numeric_constraint)
        if ("rozstaw" in q_low) or ("odstęp" in q_low) or ("spacing" in q.lower()):
            cursor.execute(
                """
                SELECT kc.title, nc.subject, nc.operator, nc.value1, nc.value2, nc.unit, kc.page_from
                FROM numeric_constraint nc
                JOIN knowledge_chunk kc ON kc.chunk_id = nc.chunk_id
                WHERE lower(nc.subject) LIKE ?
                   OR lower(COALESCE(nc.context,'')) LIKE ?
                ORDER BY kc.page_from
                LIMIT 10
                """,
                (f"%{q_low}%", f"%{q_low}%")
            )
            rows = cursor.fetchall()
            if rows:
                lines = []
                for title, subject, op, v1, v2, unit, page in rows:
                    if op == "between" and v2 is not None:
                        value_str = f"{v1}–{v2}{unit}"
                    else:
                        value_str = f"{op} {v1}{unit}"
                    lines.append(f"{title}: {subject} {value_str} (p.{page})")
                return "\n".join(lines)

    return "No information available in the database."

In [ ]:
get_information("1264-60")

In [ ]:
information_function = {
    "name": "get_information",
    "description": (
        "Retrieve technical information from the residential construction knowledge database. "
        "Use this function to get factual data about construction components, rules, or standards."
        "The query may be in English, Polish, or German; the tool will normalize terms."
    ),
    "parameters": {
        "type": "object",
        "properties": {
            "query": {
                "type": "string",
                "description": (
                    "Name or code of a construction component (e.g. 'F92-3-452', 'Wellenanker'), "
                    "or a technical topic (e.g. 'DB Fußanker M16', 'Elektryka')."
                )
            }
        },
        "required": ["query"],
        "additionalProperties": False
    }
}

tools = [{"type": "function", "function": information_function}]
tools

In [ ]:
def chat(message, history):
    history = [{"role": h["role"], "content": h["content"]} for h in history]
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL_GPT, messages=messages)
    return response.choices[0].message.content


gr.ChatInterface(fn=chat, type="messages").launch()

In [ ]:
def handle_tool_calls(message):
    responses = []

    for tool_call in message.tool_calls:
        if tool_call.function.name == "get_information":
            arguments = json.loads(tool_call.function.arguments)

            query = arguments.get("query")
            if not query:
                result = "No query provided to get_information."
            else:
                result = get_information(query)

            responses.append({
                "role": "tool",
                "tool_call_id": tool_call.id,
                "content": result
            })

    return responses

In [ ]:
def chat(message, history):
    history = [{"role":h["role"], "content":h["content"]} for h in history]
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL_GPT, messages=messages, tools=tools)


    while response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        responses = handle_tool_calls(message)
        messages.append(message)
        messages.extend(responses)
        response = openai.chat.completions.create(model=MODEL_GPT, messages=messages, tools=tools)
   
    return response.choices[0].message.content


In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()

In [ ]:
def artist(query):
    image_response = openai.images.generate(
        model="dall-e-3",
        prompt=(
            f"Technical construction drawing of {query}. "
            f"Clean line art, schematic style, neutral background, "
            f"engineering illustration, no people, no text labels."
        ),
        size="1024x1024",
        n=1,
        response_format="b64_json",
    )

    image_base64 = image_response.data[0].b64_json
    image_data = base64.b64decode(image_base64)

    return Image.open(BytesIO(image_data))

In [ ]:
image = artist("1264-60")
display(image)

In [ ]:
def talker(message):
    response = openai.audio.speech.create(
      model="gpt-4o-mini-tts",
      voice="onyx",    # Also, try replacing onyx with alloy or coral
      input=message
    )
    return response.content


In [ ]:
def handle_tool_calls_and_return_queries(message):
    responses = []
    queries = []

    for tool_call in message.tool_calls:
        if tool_call.function.name != "get_information":
            continue

        # Bezpieczne parsowanie arguments
        try:
            arguments = json.loads(tool_call.function.arguments or "{}")
        except json.JSONDecodeError:
            arguments = {}

        query = (arguments.get("query") or "").strip()

        if not query:
            result = "No query provided to get_information."
        else:
            result = get_information(query)

            # Dodaj do queries tylko jeśli to wygląda jak temat do rysunku (a nie całe zdanie)
            qlow = query.lower()
            looks_like_component = (
                re.search(r"\bf92-\d-\d+\b", qlow) is not None or
                re.search(r"\b\d{4}-\d{2}\b", qlow) is not None or  # np. 1264-60
                any(k in qlow for k in ["fußanker", "fussanker", "wellenanker", "kaiser", "dbw", "dbf"])
            )
            if looks_like_component:
                queries.append(query)

        responses.append({
            "role": "tool",
            "tool_call_id": tool_call.id,
            "content": result
        })

    return responses, queries

In [ ]:
def chat(history):
    history = [{"role": h["role"], "content": h["content"]} for h in history]
    messages = [{"role": "system", "content": system_message}] + history

    response = openai.chat.completions.create(model=MODEL_GPT, messages=messages, tools=tools)

    queries = []
    image = None

    max_loops = 5
    loops = 0

    while response.choices[0].finish_reason == "tool_calls" and loops < max_loops:
        assistant_message = response.choices[0].message
        tool_responses, new_queries = handle_tool_calls_and_return_queries(assistant_message)
        queries.extend(new_queries or [])

        messages.append(assistant_message)
        messages.extend(tool_responses)

        response = openai.chat.completions.create(model=MODEL_GPT, messages=messages, tools=tools)
        loops += 1

    reply = response.choices[0].message.content or ""
    history = history + [{"role": "assistant", "content": reply}]

    voice = talker(reply)

    if queries:
        image = artist(queries[0])

    return history, voice, image

In [64]:
def put_message_in_chatbot(message, history):
        return "", history + [{"role":"user", "content":message}]


# UI definition


with gr.Blocks() as ui:
    with gr.Row():
        chatbot = gr.Chatbot(height=500, type="messages")
        image_output = gr.Image(height=500, interactive=False)
    with gr.Row():
        audio_output = gr.Audio(autoplay=True)
    with gr.Row():
        message = gr.Textbox(label="Chat with our AI Assistant:")


# Hooking up events to callbacks


    message.submit(put_message_in_chatbot, inputs=[message, chatbot], outputs=[message, chatbot]).then(
        chat, inputs=chatbot, outputs=[chatbot, audio_output, image_output]
    )


ui.launch(inbrowser=True, auth=("ewelina", "wohnung"))


* Running on local URL:  http://127.0.0.1:7871
* To create a public link, set `share=True` in `launch()`.


DATABASE TOOL CALLED: Getting information for Wellenanker
